<a href="https://colab.research.google.com/github/ToiznerD/CloudComputing/blob/main/Project/user_interface_dor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install firebase
!pip install firebase-admin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [72]:
from zmq.constants import THREAD_AFFINITY_CPU_REMOVE
import json
from firebase import firebase
import IPython
from IPython.display import HTML
from google.colab import output, drive
from IPython.display import clear_output
from IPython.display import Markdown
import time
import ipywidgets as widgets
from google.colab.output import _publish as publish
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore, db
import matplotlib.pyplot as plt
from IPython.display import display
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

#cred = credentials.Certificate("/content/drive/My Drive/MyStyles/adminsdk.json")
#app = firebase_admin.initialize_app(cred)

def editQ():

  clear_output(wait=True)
  publish.css(style)
  


  

  def button_eventhandler(obj):
      global count, questions
      #questionsNum=-1
      # Get number of question in  and increase the amount by 1
      if questionsFromDB.value == 'Pick a question to edit':
          count += 1
          questionsNum = count

      # If it is an update to an existing question in DB
      else:
          for i,q in enumerate(questions):
            if questionsFromDB.value == q[5]:
              questionsNum = i + 1

      # Saving correct answer in DB
      correct = ''
      match(int(correct_answer.value)):
          case 1:
            correct = answer_1.value
          case 2:
            correct = answer_2.value
          case 3:
            correct = answer_3.value
          case 4:
            correct = answer_4.value

      # Insert / update a question in DB
      accessDB.update({
          questionsNum : {
            'question' : question.value,
            'correct' : correct,
            'ans1' : answer_1.value,
            'ans2' : answer_2.value,
            'ans3' : answer_3.value,
            'ans4' : answer_4.value 
          }, 'Count': {'i': count}
      })

      # Update the local array of questions
      tempQ = [answer_1.value, answer_2.value, answer_3.value,
              answer_4.value, correct, question.value]
      
      questionsTemp = list(questionsFromDB.options)
      # Update the options in questions list and print correct message according to the action(insert/update)
      if questionsNum < count:
        questions[questionsNum] = tempQ
        questionsTemp[questionsNum] = tempQ[5]
        questionsFromDB.options = questionsTemp
        label.value = 'Question "' + tempQ[5] +'" was updated successfully in DB'
      else:
        questions.append(tempQ)
        questionsTemp.append(tempQ[5])
        questionsFromDB.options = questionsTemp
        label.value = 'Question "' + tempQ[5] + '" sent to DB successfully'

      reset_fields()
      time.sleep(5)
      label.value = ''

  # Reset form's fields
  def reset_fields():
      question.value = ''
      answer_1.value = ''
      answer_2.value = ''
      answer_3.value = ''
      answer_4.value = ''
      correct_answer.value = '1'
      questionsFromDB.value = 'Pick a question to edit'


  # Fill the fields when picking a question to edit
  def fill_fields(change):
    global questions1
    # Check if default option is selected
    if change['new'] == 'Pick a question to edit':
        reset_fields()
        return

    # Find the question index and update the fields in the form
    for q in questions1:
      if q[5] == change['new']:
        answer_1.value = q[0]
        answer_2.value = q[1]
        answer_3.value = q[2]
        answer_4.value = q[3]
        question.value = q[5]
        match(str(q[4])):
          case answer_1.value:
            correct = 1
          case answer_2.value:
            correct = 2
          case answer_3.value:
            correct = 3
          case answer_4.value:
            correct = 4
        correct_answer.value = str(correct)
        break

  global questionsDict,questions1,q,count
  #@title QuestionsForm
  question = widgets.Textarea(description="Question:", layout=widgets.Layout(width='500px',height='100px'))
  answer_1 = widgets.Text(description="Answer #1:", layout=widgets.Layout(width='500px'))
  answer_2 = widgets.Text(description="Answer #2:", layout=widgets.Layout(width='500px'))
  answer_3 = widgets.Text(description="Answer #3:", layout=widgets.Layout(width='500px'))
  answer_4 = widgets.Text(description="Answer #4:", layout=widgets.Layout(width='500px'))

  correct_answer = widgets.Dropdown(
      options=['1', '2', '3', '4'],
      value='1',
      description='Correct Answer:',
      disabled=False,
      layout=widgets.Layout(width='150px'),
      style={'description_width': 'initial'})


  for qKey in questionsDict:
    if qKey != 'Count':
      q.append(questionsDict[qKey]['ans1'])
      q.append(questionsDict[qKey]['ans2'])
      q.append(questionsDict[qKey]['ans3'])
      q.append(questionsDict[qKey]['ans4'])
      q.append(questionsDict[qKey]['correct'])
      q.append(questionsDict[qKey]['question'])
      questions1.append(q)
      q = []
    else:
      count = int(questionsDict[qKey]['i'])

  q.append('Pick a question to edit')
  for qu in questions1:
    q.append(qu[5])

  questionsFromDB = widgets.Dropdown(
      options = q
  )

  add_edit_btn = widgets.Button(description='Add / Edit Question')
  add_edit_btn.on_click(button_eventhandler)

  label = widgets.Label(value='')

  questionsFromDB.observe(fill_fields)

  logo()
  
  display(IPython.display.HTML('''
    <head>
      <h2>Edit</h2>
    </head>
  '''))

  exitBt = widgets.Button(description="Back", layout={'width': '250px','height': '80px', 'margin': '10px'})
  exitBt.add_class("button")
  exitBt.on_click(lambda button: beginGame())

  container = widgets.VBox([question, answer_1, answer_2, answer_3, answer_4, correct_answer, questionsFromDB, add_edit_btn, label,exitBt], layout=widgets.Layout(align_items='center',justify_content='flex-start'))
  display(container)

# Get questions from DB
accessDB = db.reference('Questions/', app=firebase_admin.get_app(name="abc"))

questionsDict = accessDB.get()
questions1 = []
q = []
count = 0 

def logo():
  image_widget = widgets.Image(value=open('/content/drive/My Drive/MyStyles/logo.jpeg', 'rb').read(), format='jpeg')
  image_widget.layout.width = '400px'
  image_widget.layout.height = 'auto'
  imgContainer = widgets.VBox([image_widget], layout=widgets.Layout(align_items='center'))
  display(imgContainer)



def userNav():
  #navbar
  navbarLabel1 = widgets.HTML(f'Hello {user["username"]}!')
  navbarLabel1.add_class('navbar')
  navbarLabel2 = widgets.HTML(f'Total Points: {user["points"]}')
  navbarLabel2.add_class('navbar')
  navbarContainer = widgets.VBox([navbarLabel1, navbarLabel2], layout=widgets.Layout(align_items='center'))
  display(navbarContainer)

# Get style
drive.mount('/content/drive', force_remount=True)
with open('/content/drive/My Drive/MyStyles/style.css') as fid:
  style = fid.read()


questions = []
rAnswers=0
wAnswers=0
qnum = 0
qnumLabel = None
questionLabel = None
container = None
user = {}
################# LOGIN
def loginButton_eventhandler(obj):
  global user
  user = db.reference('Users/', app=firebase_admin.get_app(name="abc")).child(username.value).get()

  with open('/content/drive/My Drive/MyStyles/spinner.gif', 'rb') as f:
      img = f.read()
  #   create loading bar widget, ready to display when running long function
  loading_bar = widgets.Image(value=img, layout=widgets.Layout(width="25px", height="auto"))
  spinnerContainer = widgets.VBox([loading_bar], layout=widgets.Layout(width='100%', align_items='center'))
  display(spinnerContainer)
  time.sleep(7)
  spinnerContainer.children = []

  # Check if user is exists in DB
  if user:
      if user['password'] == password.value:
        label.value = 'You have successfully logged in!'

        time.sleep(2)
        beginGame()
      else:
        label.value = 'Wrong details or user is not exists'

publish.css(style)
logo()


# User input
userLabel = widgets.Label(value='User Name:', layout=widgets.Layout(width='150px'))
userLabel.add_class('login')
username = widgets.Text(layout=widgets.Layout(width='250px'))
userHbox = widgets.HBox([userLabel, username])

# Password input
pwLabel = widgets.Label(value='Password:', layout=widgets.Layout(width='150px'))
pwLabel.add_class('login')
password = widgets.Password(layout=widgets.Layout(width='250px'))
password.add_class('login')
pwHbox = widgets.HBox([pwLabel, password])

# Login button
btn = widgets.Button(description='Login')
btn.on_click(loginButton_eventhandler)
btn.add_class('button')

# Error label
label = widgets.Label(value='', layout=widgets.Layout(height='100px'))
label.add_class('login')

container = widgets.VBox([userHbox, pwHbox, btn, label], layout=widgets.Layout(width='100%', align_items='center'))
display(container)



################ main menu
def beginGame():
  clear_output(wait=True)
  publish.css(style)
  logo()
  userNav()
  display(IPython.display.HTML('''
    <head>
      <h2>Lets Play</h2>
    </head>
  '''))
  playAgain = widgets.Button(description="Start!")
  playAgain.add_class("button")
  playAgain.on_click(lambda button: loadGame())

  edit = widgets.Button(description="Edit Q/A")
  edit.add_class("button")
  edit.on_click(lambda button: editQ())

  container = widgets.VBox([playAgain,edit], layout=widgets.Layout(width='100%', align_items='center'))
  display(container)



################# LOAD GAME
def loadGame():
  clear_output(wait=True)
  ref = db.reference('Questions/', app=firebase_admin.get_app(name="abc")).get()
  for key, value in ref.items():
    if key != 'Count':
      questions.append(value)

  # Loading window

  publish.css(style)
  logo()
  userNav()
  display(IPython.display.HTML('''
  <html>
    <head>
      <h2>level 1</h2>
    </head>
    <body>

      <!-- Loading bar -->
      <div id="loading-bar">
        <div id="loading-progress"></div>
      </div>

      <!-- JavaScript code to simulate loading the game -->
      <script>
        var progress = 0;
        var loading = setInterval(function() {
          progress += Math.floor(Math.random() * 10); // simulate progress
          document.getElementById('loading-progress').style.width = progress + '%';
          if (progress >= 100) {
            clearInterval(loading); // stop the interval when progress reaches 100%
          }
        }, 500);
      </script>
    </body>
  </html>
  '''))

  time.sleep(10)

  # Clear loading window
  startGame()


################# START GAME


def startGame():
  clear_output(wait=True)
  ## START GAME UTILS ##
  def reloadGame():
    global wAnswers, rAnswers, qnum, questions
    wAnswers = 0
    rAnswers = 0
    qnum = 0
    questions = []
    loadGame()

  def checkAnswer(ans, buttons):
    cButton = None
    global qnum

    # Check if container is visible
    if container.layout.display == 'none':
      return

    # Find the correct answer
    for b in buttons:
      if b.description == questions[qnum]['correct']:
        cButton = b
    
    # Validate the clicked answer
    if ans.description == cButton.description:
      ans.add_class('correctAnswer')
      global rAnswers
      rAnswers += 1
    else:
      ans.add_class('wrongAnswer')
      cButton.add_class('correctAnswer')
      global wAnswers
      wAnswers += 1
    
    time.sleep(3)
    
    qnum += 1
    if qnum == 5: # in case of last question
      container.layout.display = 'none'
      rAnswersLabel = widgets.HTML(value=f'Correct Answers: {rAnswers}')
      rAnswersLabel.add_class('h5')
      wAnswersLabel = widgets.HTML(value=f'Wrong Answers: {wAnswers}')
      wAnswersLabel.add_class('h4')
      bonusPtsLabel = widgets.HTML(value=f'You get {rAnswers*10} new points!')
      bonusPtsLabel.add_class('h6')
      playAgain = widgets.Button(description="Play Again!", layout={'width': '350px','height': '100px', 'margin': '10px'})
      playAgain.add_class("button")
      playAgain.on_click(lambda button: reloadGame())



      # Define the values as variables
      values = [rAnswers, wAnswers]
      labels = ['Correct Answers', 'Wrong Answers']

      # Create the pie chart
      fig, ax = plt.subplots()

      #Set the background color for the entire plot area
      ax.set_facecolor('#032f40')  # Set the background color of the axes

      # Plot the pie chart
      _, texts, autotexts = ax.pie(values, labels=labels, autopct='%1.1f%%')

      # Set the background color for the entire figure
      plt.gcf().set_facecolor('#032f40')  # Set the background color of the figure

      # Optional: Set title and adjust layout
      ax.set_title('Answers Summary', color='white')
      fig.tight_layout()

      ax.tick_params(labelcolor='white')

      # Set color for pie chart labels
      for text in texts + autotexts:
          text.set_color('white')

     

      img = fig.savefig('pie_chart.png')



      plt.close(fig)

      image_widget = widgets.Image(value=open('pie_chart.png', 'rb').read())

      image_widget.layout.width = '400px'
      image_widget.layout.height = '300px'
      
      Containerimage = widgets.VBox([image_widget], layout=widgets.Layout(align_items='center', height="320px"))
      display(Containerimage)


      answersContainer = widgets.VBox([rAnswersLabel, wAnswersLabel, bonusPtsLabel, playAgain], layout=widgets.Layout(align_items='center', height="500px"))
      display(answersContainer)

      


      user['points'] += rAnswers*10
      db.reference('Users/', app=firebase_admin.get_app(name="abc")).child(user['username']).update(user)




      return
    
    # Present next question
    questionLabel.value = questions[qnum]['question']
    qnumLabel.value = f'Question #{qnum+1}'
    for i,b in enumerate(buttons):
      ansStr = ansStr = f'ans{i+1}'
      b.description = questions[qnum][ansStr]
      b.remove_class("wrongAnswer")
      b.remove_class("correctAnswer")
      b.add_class("button")
  
  ## END UTILS ##

  publish.css(style)

  # Present logo
  logo()
  userNav()

  button_layout = {'width': '350px','height': '100px', 'margin': '10px'}
  buttons = []
  for i in range(1,5):
    ansStr = f'ans{i}'
    b = widgets.Button(description=questions[qnum][ansStr], layout=button_layout)
    b.add_class("button")
    buttons.append(b)

  button_container = widgets.HBox(children=buttons)

  # Add listeners
  for b in buttons:
    b.on_click(lambda button: checkAnswer(button, buttons))


  # Wrap the button container in a VBox container and center it horizontally
  qnumLabel = widgets.HTML(value=f'Question #{qnum+1}')
  qnumLabel.add_class("h2")
  questionLabel = widgets.HTML(value=questions[0]['question'])
  questionLabel.add_class("h2")

  exitBt = widgets.Button(description="Exit", layout={'width': '250px','height': '80px', 'margin': '10px'})
  exitBt.add_class("button")
  exitBt.on_click(lambda button: beginGame())

  container = widgets.VBox([qnumLabel, questionLabel, button_container,exitBt], layout=widgets.Layout(align_items='center'))
  display(container)


In [3]:

import firebase_admin
from firebase_admin import credentials, db

# Initialize the Firebase Admin SDK
cred = credentials.Certificate('/content/drive/My Drive/MyStyles/adminsdk.json')
firebase_admin.initialize_app(cred,{'databaseURL': 'https://cloudcomputing-e1f5d-default-rtdb.firebaseio.com/'}, name="abc")


print(db.reference('Users/', app=firebase_admin.get_app(name="abc")).get())


{'amit': {'password': '123', 'points': 590, 'username': 'amit'}, 'benshi': {'password': '123', 'points': 390, 'username': 'benshi'}, 'dor': {'password': '123', 'points': 600, 'username': 'dor'}, 'raz': {'password': '123', 'points': 460, 'username': 'raz'}}
